In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 НЕ выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from IPython.display import display

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler, OneHotEncoder, RobustScaler
from sklearn.preprocessing import LabelEncoder, QuantileTransformer
from sklearn.model_selection import train_test_split, cross_validate, KFold, GridSearchCV, StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsRegressor 

from sklearn.linear_model import ElasticNet, Ridge, Lasso, Lars, LinearRegression
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.compose import TransformedTargetRegressor
# pip install category-encoders

import category_encoders as ce

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.metrics import classification_report, mean_absolute_percentage_error

from sklearn.svm import SVR

### Область работы 2 (поиск  модели .... )

In [2]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели 
# должен быть понятен и очевиден отбор параметров модели
# оставляйте свои комментарии и разъяснения
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# ЗАПРЕЩАЕТСЯ ИСПОЛЬЗОВАТЬ ТЕСТОВЫЙ НАБОР
#
# Путь к тренировочному набору
# 

path_train = 'train.csv' # содержит только имя файла, без имен папок !!!

In [3]:
car = pd.read_csv(path_train)

In [5]:
display(car.info())
car.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38005 entries, 0 to 38004
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   company             38005 non-null  object 
 1   model               38005 non-null  object 
 2   year                38005 non-null  int64  
 3   condition           38005 non-null  object 
 4   mileage(km)         38005 non-null  float64
 5   fuel                38005 non-null  object 
 6   volume(cm3)         38005 non-null  float64
 7   color               38005 non-null  object 
 8   transmission        38005 non-null  object 
 9   drive_unit          38005 non-null  object 
 10  vehicle_size_class  34434 non-null  object 
 11  priceUSD            38005 non-null  int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 3.5+ MB


None

,company,model,year,condition,mileage(km),fuel,volume(cm3),color,transmission,drive_unit,vehicle_size_class,priceUSD
0,renault,laguna,2000,with mileage,230000.0,petrol,1800.0,blue,mechanics,front-wheel drive,D,2300
1,chrysler,pacifica,2005,with mileage,310000.0,petrol,3500.0,black,auto,front-wheel drive,NaN,6300
2,mercedes-benz,c-klass,2010,with mileage,115000.0,petrol,1800.0,gray,auto,rear drive,D,11200
3,hyundai,accent,2017,with mileage,45500.0,petrol,1400.0,gray,auto,front-wheel drive,B,10886
4,renault,kadjar,2016,with mileage,139000.0,diesel,1500.0,red,auto,front-wheel drive,J,15300


In [6]:
display(car.isna().sum())
display(car.duplicated().sum())

company                  0
model                    0
year                     0
condition                0
mileage(km)              0
fuel                     0
volume(cm3)              0
color                    0
transmission             0
drive_unit               0
vehicle_size_class    3571
priceUSD                 0
dtype: int64

39

In [7]:
car.drop_duplicates(inplace = True)

In [8]:
display(car['vehicle_size_class'].unique())
car['vehicle_size_class'].nunique()

array(['D', nan, 'B', 'J', 'C', 'E', 'M', 'F', 'S', 'A'], dtype=object)

9

In [9]:
for i in car.columns:
    display(i)
    display(car[i].unique())
    display(car[i].nunique())

'company'

array(['renault', 'chrysler', 'mercedes-benz', 'hyundai', 'ford',
       'chevrolet', 'peugeot', 'volkswagen', 'audi', 'mazda', 'opel',
       'nissan', 'toyota', 'lada-vaz', 'daewoo', 'land-rover', 'skoda',
       'bmw', 'mitsubishi', 'uaz', 'honda', 'fiat', 'citroen', 'proton',
       'saab', 'volvo', 'daihatsu', 'kia', 'seat', 'jeep', 'dacia',
       'lancia', 'suzuki', 'jaguar', 'buick', 'lexus', 'gaz', 'moskvich',
       'alfa-romeo', 'subaru', 'ssangyong', 'isuzu', 'infiniti', 'chery',
       'zaz', 'acura', 'tata', 'eksklyuziv', 'smart', 'rover', 'lifan',
       'luaz', 'vortex', 'porsche', 'scion', 'mercury', 'great-wall',
       'mini', 'saipa', 'geely', 'iran-khodro', 'cadillac', 'pontiac',
       'lincoln', 'brilliance', 'bentley', 'changan', 'saturn', 'gmc',
       'plymouth', 'izh', 'ravon', 'faw', 'maserati', 'oldsmobile',
       'zotye', 'datsun', 'wartburg', 'mg', 'asia', 'aston-martin',
       'hafei', 'haval', 'dong-feng', 'raf', 'fso', 'byd', 'aro',
       'shanghai-

91

'model'

array(['laguna', 'pacifica', 'c-klass', 'accent', 'kadjar', 'escort',
       'volt', 'e-klass', 'aveo', '308', 'golf', '5008', 'a6', '3008',
       '406', '626', 'm-klass', 'a4', 'scenic', 'xedos-6', 'caravelle',
       'zafira', 'x-trail', 'avensis', '605', 'a8', 'vesta', 'corolla',
       '100', 'leganza', 'range-rover-sport', 'jetta', 'fabia',
       '5-seriya', 'a7', 'passat', 'galant', '80', '469', 'crosstour',
       'vectra', 'qashqai', 'x5', 'bravo', '2109', 'kodiaq', 'octavia',
       'sonata', 'accord', 'pathfinder', 'combo', 'granta', 's-klass',
       '207', 'xantia', 'galaxy', 'cr-v', '3-seriya', 'safrane',
       'touareg', 'ulysse', 'vaneo', 'note', 'santa-fe', 'x6', 'c4',
       'vivaro', 'persona', 'almera-tino', 'space-wagon', 's-max',
       'brava', 'sharan', '9-5', '440', 'yrv', 'sportage', 'largus',
       'c-max', 's80', '607', 'superb', 'berlingo', 'omega', 'alhambra',
       '19', 'z4', 'highlander', 'millenia', 'touran', 'outlander',
       'marea', 'land-crui

...

array(['D', nan, 'B', 'J', 'C', 'E', 'M', 'F', 'S', 'A'], dtype=object)

9

'priceUSD'

array([ 2300,  6300, 11200, ...,  2180, 13370, 31950])

2467

In [11]:
print (car[["color", "priceUSD"]].groupby(['color'], as_index=False).mean(), '\n')
print (car[["transmission", "priceUSD"]].groupby(['transmission'], as_index=False).mean(), '\n')
print (car[["vehicle_size_class", "priceUSD"]].groupby(['vehicle_size_class'], as_index=False).mean(), '\n')
print (car[["drive_unit", "priceUSD"]].groupby(['drive_unit'], as_index=False).mean(), '\n')
print (car[["condition", "priceUSD"]].groupby(['condition'], as_index=False).mean(), '\n')
print (car[["fuel", "priceUSD"]].groupby(['fuel'], as_index=False).mean(), '\n')

       color      priceUSD
0      black  10732.164791
1       blue   5200.284267
2      brown  12259.837284
3   burgundy   4508.072727
4       gray   8504.864691
5      green   3078.063534
6     orange   6426.782609
7      other   6740.271861
8     purple   5314.420690
9        red   4594.362561
10    silver   6205.401915
11     white   9563.818546
12    yellow   6657.468293 

  transmission      priceUSD
0         auto  12379.201697
1    mechanics   4638.534085 

  vehicle_size_class      priceUSD
0                  A   4233.085809
1                  B   4271.694350
2                  C   4841.692578
3                  D   5368.147339
4                  E   6232.257042
5                  F  10918.865052
6                  J  14437.927492
7                  M   6941.856440
8                  S  17212.316794 

                   drive_unit      priceUSD
0             all-wheel drive  16861.264413
1           front-wheel drive   5617.387003
2  part-time four-wheel drive  12341.165283
3  

In [37]:
y = np.array(car.priceUSD)
X = car.drop(columns=['priceUSD'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [48]:
transmission_map = [{
    'col':'transmission',
    'mapping':{'auto':12, 'mechanics':4}
    }]

transmission_transformer = Pipeline(steps = [
    ('ce',ce.OrdinalEncoder(mapping = transmission_map)),
    ('scaler', StandardScaler())
    ])

####################################

drive_unit_map = [{
    'col':'drive_unit',
    'mapping':{'all-wheel drive':16, 'part-time four-wheel drive':12, 'rear drive':6, 'front-wheel drive':5}
    }]

drive_unit_transformer = Pipeline(steps = [
    ('ce',ce.OrdinalEncoder(mapping = drive_unit_map)),

    ('scaler', StandardScaler())
    ])

####################################

condition_map = [{
    'col':'condition',
    'mapping':{'with mileage':75, 'with damage':26, 'for parts':6}
    }]

condition_transformer = Pipeline(steps = [
    ('ce',ce.OrdinalEncoder(mapping = condition_map)),
    ('scaler', StandardScaler())
    ])

####################################

fuel_map = [{
    'col':'fuel',
    'mapping':{'diesel':4, 'petrol':3}
    }]

fuel_transformer = Pipeline(steps = [
    ('ce',ce.OrdinalEncoder(mapping = fuel_map)),
    ('scaler', StandardScaler())
    ])

####################################

color_map = [{
    'col':'color',
    'mapping':{'black':10, 'blue':5, 'brown':12, 'burgundy':4, 'gray':8, 
               'green':3, 'orange':6, 'other':6, 'purple':5, 'red':4,
              'silver':6, 'white':9, 'yellow':6}
    }]

color_transformer = Pipeline(steps = [
    ('ce',ce.OrdinalEncoder(mapping = color_map)),
    ('scaler', MinMaxScaler())
    ])

####################################

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output = False, handle_unknown='ignore').set_output(transform='pandas')),
    ('scaler', StandardScaler())
])

####################################

vehicle_size_class_map = [{
    'col':'vehicle_size_class',
    'mapping':{'A':4, 'B':4, 'C':5, 'D':5, 'E':6, 
               'F':10, 'J':14, 'M':7, 'S':17
              }
    }]

vehicle_size_class_transformer = Pipeline(steps = [
    ('imputer_CT', SimpleImputer(strategy='most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping = vehicle_size_class_map)),
    ('scaler', StandardScaler())
    ])

In [39]:
features_cat = ['company', 'model']
features_num_1 = ['year']
features_num_2 = ['mileage(km)']
features_num_3 = ['volume(cm3)']

In [40]:
CT = ColumnTransformer([
        ("features_cat", categorical_transformer, features_cat),
        ("features_num_1", RobustScaler(), features_num_1),
        ("features_num_2", MinMaxScaler(), features_num_2),
        ("features_num_3", StandardScaler(), features_num_3),
        ("condition_map", condition_transformer, ['condition']),
        ("fuel_map", fuel_transformer, ['fuel']),
        ("color_map", color_transformer, ['color']),
        ("transmission_map", transmission_transformer, ['transmission']),
        ("drive_unit_map", drive_unit_transformer, ['drive_unit']),
        ("vehicle_size_class_map", vehicle_size_class_transformer, ['vehicle_size_class']),
        ],remainder='drop')

display(CT)

CT.set_output(transform ='pandas')

res_ct = CT.fit_transform(X_train)
pd.DataFrame(res_ct).head()

ColumnTransformer(transformers=[('features_cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['company', 'model']),
                                ('features_num_1', RobustScaler(), ['year']),
                                ('features_num_2', MinMaxScaler(),
                                 ['mileage(km)']),
                                ('features_num_3', St...
front-wheel drive              5
dtype: int64}])),
                                                 ('scaler', StandardScaler())]),
                                 ['drive_unit']),
                                ('vehicle_size_class_map',
                                 Pipeline(steps=[('imputer_CT',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': A     4
B     4
C     5
D     5
E     6
F    10
J    14
M     7
S    17
dtype: int64}])),
                                                 ('scaler', StandardScaler())]),
                                 ['vehicle_size_class'])])

,features_cat__company_acura,features_cat__company_alfa-romeo,features_cat__company_aro,features_cat__company_asia,features_cat__company_aston-martin,features_cat__company_audi,features_cat__company_bentley,features_cat__company_bmw,features_cat__company_buick,features_cat__company_byd,...,features_cat__model_zeta,features_num_1__year,features_num_2__mileage(km),features_num_3__volume(cm3),condition_map__condition,fuel_map__fuel,color_map__color,transmission_map__transmission,drive_unit_map__drive_unit,vehicle_size_class_map__vehicle_size_class
36381,-0.043184,-0.06368,-0.005926,-0.005926,-0.005926,-0.277507,-0.014518,-0.274634,-0.041941,-0.008381,...,-0.017781,0.416667,0.024700,1.189234,0.125843,1.357935,0.777778,1.333195,2.538068,2.057778
12057,-0.043184,-0.06368,-0.005926,-0.005926,-0.005926,-0.277507,-0.014518,-0.274634,-0.041941,-0.008381,...,-0.017781,0.416667,0.002800,-0.650378,0.125843,1.357935,0.777778,-0.750078,-0.485841,-0.544358
29318,-0.043184,-0.06368,-0.005926,-0.005926,-0.005926,-0.277507,-0.014518,-0.274634,-0.041941,-0.008381,...,-0.017781,1.000000,0.013785,-0.109316,0.125843,1.357935,1.000000,-0.750078,-0.485841,-0.544358
32361,-0.043184,-0.06368,-0.005926,-0.005926,-0.005926,-0.277507,-0.014518,-0.274634,-0.041941,-0.008381,...,-0.017781,-0.833333,0.000035,-0.542165,0.125843,1.357935,0.666667,-0.750078,-0.485841,-0.544358
36205,-0.043184,-0.06368,-0.005926,-0.005926,-0.005926,3.603507,-0.014518,-0.274634,-0.041941,-0.008381,...,-0.017781,0.416667,0.011200,0.756384,0.125843,-0.736412,0.777778,1.333195,-0.485841,-0.255232


In [45]:
start = time.time()

pipe = Pipeline([('preprocessing', CT), 
                 ('classifier', KNeighborsRegressor())])

regressors = [Ridge(), ElasticNet(), Lars(), Lasso(), KNeighborsRegressor(), RandomForestRegressor(),
             LinearRegression(), GradientBoostingRegressor(), SVR()]
param_grid = [
    {
        'classifier': regressors
    },
]

grid = GridSearchCV(pipe, param_grid, cv = KFold(5), return_train_score = True,
                    scoring = 'neg_mean_absolute_percentage_error', n_jobs=-1)
grid.fit(X_train[:8000], y_train[:8000])

print(f"цикл пройден за {round((time.time() - start)/60, 1)} минут")

/Users/y1ov/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_least_angle.py:645: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 5 iterations, i.e. alpha=1.319e+03, with an active set of 5 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
/Users/y1ov/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_least_angle.py:645: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 22 iterations, i.e. alpha=5.113e+02, with an active set of 22 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
/Users/y1ov/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_least_angle.py:645: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 41 iterations, i.e. alpha=2.899e+02, with an active set of 41 regressors, and the sm

/Users/y1ov/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.690e+08, tolerance: 4.131e+07
  model = cd_fast.enet_coordinate_descent(
/Users/y1ov/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.180e+08, tolerance: 4.531e+07
  model = cd_fast.enet_coordinate_descent(
/Users/y1ov/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality 

цикл пройден за 1.1 минут


In [46]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

,5,7,4,1,3,0,8,6,2
mean_fit_time,9.671393,4.284435,0.15688,0.37921,7.794861,0.36371,15.137185,0.875908,0.61269
std_fit_time,0.149853,0.100152,0.011402,0.031365,0.306094,0.031909,0.064485,0.025304,0.02193
mean_score_time,0.089541,0.031805,0.723023,0.049179,0.030896,0.034949,5.211677,0.027424,0.036829
std_score_time,0.014284,0.010992,0.07346,0.014745,0.006565,0.006518,0.18438,0.006093,0.010406
param_classifier,RandomForestRegressor(),GradientBoostingRegressor(),KNeighborsRegressor(),ElasticNet(),Lasso(),Ridge(),SVR(),LinearRegression(),Lars()
params,{'classifier': RandomForestRegressor()},{'classifier': GradientBoostingRegressor()},{'classifier': KNeighborsRegressor()},{'classifier': ElasticNet()},{'classifier': Lasso()},{'classifier': Ridge()},{'classifier': SVR()},{'classifier': LinearRegression()},{'classifier': Lars()}
split0_test_score,-0.268878,-0.36741,-0.380157,-0.59059,-0.707844,-0.710411,-1.484518,-2545452466809.607422,-39026852537518.609375
split1_test_score,-0.291251,-0.39503,-0.432143,-0.603485,-0.729765,-0.730624,-1.514877,-120828936361.21312,-286799876720.502625
split2_test_score,-0.288923,-0.382868,-0.402622,-0.660154,-0.770198,-0.772032,-1.646393,-365372362785.038696,-3513412327671412.0
split3_test_score,-0.247891,-0.340994,-0.380707,-0.594065,-0.684986,-0.685335,-1.486842,-104442704495.328949,-22631662960014.40625


# RF

In [47]:
best_pipe = TransformedTargetRegressor(
                        regressor = Pipeline(steps = [
                            ('preproc',CT),
                            ('estimator',RandomForestRegressor(n_estimators = 1000, random_state=42,
                                                               n_jobs = -1))
                        ]),
    transformer = QuantileTransformer(n_quantiles = 1000 , output_distribution = 'normal')
)

best_pipe.fit(X_train, y_train)
y_pred_best = best_pipe.predict(X_test)
y_pred_best_train = best_pipe.predict(X_train)
print('MAPE на трэйне: ',mean_absolute_percentage_error(y_train, y_pred_best_train))
print('MAPE на тесте: ',mean_absolute_percentage_error(y_test, y_pred_best))

MAPE на трэйне:  0.07474096371407005
MAPE на тесте:  0.2169144272454181


# GB

In [50]:
best_pipe = TransformedTargetRegressor(
                        regressor = Pipeline(steps = [
                            ('preproc',CT),
                            ('estimator',GradientBoostingRegressor(n_estimators = 1000))
                        ]),
    transformer = QuantileTransformer(n_quantiles = 1000 , output_distribution = 'normal')
)

best_pipe.fit(X_train, y_train)
y_pred_best = best_pipe.predict(X_test)
y_pred_best_train = best_pipe.predict(X_train)
print('MAPE на трэйне: ',mean_absolute_percentage_error(y_train, y_pred_best_train))
print('MAPE на тесте: ',mean_absolute_percentage_error(y_test, y_pred_best))

MAPE на трэйне:  0.20187034162802503
MAPE на тесте:  0.21633706598270802


In [100]:
best_pipe = TransformedTargetRegressor(
                        regressor = Pipeline(steps = [
                            ('preproc',CT),
                            ('estimator',GradientBoostingRegressor(n_estimators = 1000,
                                                                  max_depth = 7,
                                                                learning_rate = 0.04,
                                                                  subsample=0.6,
                                                                   min_samples_split = 15,
                                                                  random_state=42))
                        ]),
    transformer = QuantileTransformer(n_quantiles = 1000 , output_distribution = 'normal')
)

best_pipe.fit(X_train, y_train)
y_pred_best = best_pipe.predict(X_test)
y_pred_best_train = best_pipe.predict(X_train)
print('MAPE на трэйне: ',mean_absolute_percentage_error(y_train, y_pred_best_train))
print('MAPE на тесте: ',mean_absolute_percentage_error(y_test, y_pred_best))

MAPE на трэйне:  0.17660019572743604
MAPE на тесте:  0.20547274768355034
